In [1]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'processed'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [3]:
ct = pd.read_csv(src / 'metrics-from-test-data' / 'ct.csv', index_col='date', parse_dates=True)
# ct_by_awh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-awh.csv')
ct_by_adh = pd.read_csv(
    src / 'metrics-from-test-data' / 'ct-by-adh.csv',
    index_col='date',
    dtype={'addr_dist_home': 'int'},
    parse_dates=True)
# ct_by_sex = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-sex.csv')
# ct_by_ag = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-ag.csv')
addiv = pd.read_csv(
    ref / 'addiv.csv',
    usecols=['id_addiv', 'name_addiv_2'],
    dtype={'id_addiv': 'int'}
)
pop = pd.read_csv(
    ref / 'pop_1.csv',
    usecols=['id_addiv', 'pop'],
    dtype={'id_addiv': 'int'})

In [7]:
ct
# addiv.info()
# pop.info()

,no_test,no_test_pos,no_test_rollingsum7d,no_test_rollingmean7d,no_test_pos_rollingsum7d,no_test_pos_rollingmean7d,pct_test_pos_per7d,ct,situational_level
date,,,,,,,,,
2021-05-27,2965.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-28,3495.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-29,5083.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-30,6828.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31,5514.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2021-09-24,2299354.0,11014.0,8137745.0,1.162535e+06,69697.0,9956.714286,0.008565,1.0,2.0
2021-09-25,1817203.0,7634.0,9881665.0,1.411666e+06,68593.0,9799.000000,0.006941,1.0,2.0
2021-09-26,2114164.0,8282.0,11896256.0,1.699465e+06,68319.0,9759.857143,0.005743,1.0,2.0


In [5]:
tphcm = {
    'addr_dist_home':[79],
    'no_test': [ct[['no_test', 'no_test_pos']].sum()[0]],
    'no_test_pos': [ct[['no_test', 'no_test_pos']].sum()[1]]
}

df_tphcm = pd.DataFrame(tphcm)

df = (
    ct_by_adh[['addr_dist_home', 'no_test', 'no_test_pos']]
#     .query('date_sample == "2021-08-05"')
    .groupby('addr_dist_home')[['no_test', 'no_test_pos']]
    .sum()
#     .to_frame(name='no_test_cumsum')
    .reset_index()
    .append(df_tphcm)
    .sort_values('no_test')
    .merge(
        addiv,
        how = 'left',
        left_on = 'addr_dist_home',
        right_on = 'id_addiv'
    )
    .merge(
        pop,
        how = 'left',
        left_on = 'id_addiv',
        right_on = 'id_addiv'
    )
    .drop(columns={'addr_dist_home', 'id_addiv'})
    .rename(columns={'name_addiv_2': 'addr_dist_home'})
)

df['pct_test_pos'] = df['no_test_pos'] / df['no_test']

df['predicted_inc'] = df['pop'] * df['pct_test_pos']

df.to_csv(src / 'predicted-inc-from-test-data' / 'predicted-inc.csv', index=False)
df

,no_test,no_test_pos,addr_dist_home,pop,pct_test_pos,predicted_inc
0,4626.0,1406.0,QUAN 09,397006,0.303934,120663.734544
1,9953.0,3385.0,QUAN 02,180275,0.340098,61311.250377
2,10809.0,3656.0,QUAN THU DUC,592686,0.338237,200468.129892
3,119713.0,2362.0,HUYEN CAN GIO,71526,0.019731,1411.245328
4,218173.0,15011.0,QUAN 01,142625,0.068803,9813.056038
5,275068.0,12817.0,QUAN 03,190375,0.046596,8870.666072
6,280824.0,15749.0,QUAN 05,159073,0.056081,8921.034801
7,288906.0,22568.0,QUAN 10,234819,0.078115,18342.973812
8,304815.0,7089.0,QUAN PHU NHUAN,163961,0.023257,3813.196624
9,360799.0,19237.0,QUAN 04,175329,0.053318,9348.152220


In [6]:
# df = (
#     ct_by_adh[['addr_dist_home', 'date_sample', 'no_test', 'no_positive']]
#     .groupby(['addr_dist_home', 'date_sample']).sum()
#     .groupby(level=0).cumsum().reset_index()
#     .merge(
#         addiv,
#         how = 'left',
#         left_on = 'addr_dist_home',
#         right_on = 'name_addiv_2'
#     )
#     .merge(
#         pop,
#         how = 'left',
#         left_on = 'id_addiv',
#         right_on = 'id_addiv'
#     )
#     .drop(columns={'id_addiv', 'name_addiv_2'})
# )

# df['pct_positive'] = df['no_positive'] / df['no_test']

# df['predicted_inc'] = df['pop'] * df['pct_positive']
# # foo[foo.addr_dist_home == 'HUYEN BINH CHANH'].tail(10)
# df